## Purpose

This was a very quick weekend project to get my hands dirty with transformers and get something very basic working. First time playing with this and coded by teaching myself basic syntax using Hugging Face tutorials and ChatGPT.

The purpose of this notebook is to:
* Familiarise myself with the transformer and torch interface
* Construct very simple examples to demonstrate the shortcomings of LLMs and the impact of good prompting/priming first hand.
* See how truly (and surprisingly) bad transformers are

Note to self for further mini-projects:
* Try and create a basic transformer from scratch by assembling the different attention heads, MLP's, Normalisation, etc.
* Try and train a toy transformer that can just add numbers (https://arxiv.org/abs/2301.05217).

## Familiarising with the objects

* Looking at dictionary
* Encoding and decoding text
* Trying to generate a prompt and get a response back


In [12]:
#libraries
import transformers as tf
import torch as torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model directly
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

#Still sticking with using CPU for the small model

In [2]:
#Sample coding and decoding of text
sample_encoded = tokenizer.encode("Hello, what is your name? My name is Preetham")
print(sample_encoded)
sample_decoded = tokenizer.decode(sample_encoded)
print(sample_decoded)

#Looking at the dictionary
print(tokenizer.vocab_size)
sorted_vocab = sorted(list(tokenizer.vocab.items()), key=lambda n:n[1])
print(sorted_vocab[0:10:])
print(sorted_vocab[-1:-10:])


[15496, 11, 644, 318, 534, 1438, 30, 2011, 1438, 318, 350, 2871, 2763]
Hello, what is your name? My name is Preetham
50257
[('!', 0), ('"', 1), ('#', 2), ('$', 3), ('%', 4), ('&', 5), ("'", 6), ('(', 7), (')', 8), ('*', 9)]
[]


## Looking at a sample prompt generation

**Aim:**
In this section, I am simply exploring how a sample prompt can be used to generate a response.

Here I just want to restrict it to simple Yes/No style.

**Observations**: Here not much of a difference between the Yes or No response. Either it does not understand the task or it is just bad prompting. Even the generated text does not ask Yes/No questions as a follow up.


In [15]:
prompt = 'You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. Question: Do all humans have three heads? Answer:'

input_ids = tokenizer([prompt], return_tensors="pt")

output_sequences = model.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)

#Just looking at what it wishes to produce
generated_sequence = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)

#Now looking at the logits for the next token that it wants to produce
with torch.no_grad():
    outputs = model(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]

# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)

#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. Question: Do all humans have three heads? Answer: No. Question: Why does the world have so many different species? Answer: Because they have different
(' Yes', 3363): -130.54352, 0.31525
(' No', 1400): -130.56625, 0.30817
(' I', 314): -133.50876, 0.01625
(' yes', 3763): -133.74930, 0.01278
(' "', 366): -133.85109, 0.01154
(' The', 383): -133.86220, 0.01141
(' Not', 1892): -133.87099, 0.01131
(' Humans', 27411): -134.02776, 0.00967
(' You', 921): -134.08125, 0.00917
(' None', 6045): -134.15588, 0.00851


## Few-shot learning attempt
**Aim:**
Now I am looking at how few-shot learning improves the chances of getting a 'No' Token.

**Observations**: Definitely makes a big difference where it reduces the probability it assigns to words other than just Yes/No. Also, it the sentence it generates now is more in line with the sample questions/answers.

In [16]:
prompt = 'You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky green? Answer:No Question: Do all humans have three heads? Answer:'

input_ids = tokenizer([prompt], return_tensors="pt")

output_sequences = model.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky green? Answer:No Question: Do all humans have three heads? Answer:Yes Question: Is the sun blue? Answer:No Question: Is the moon blue? Answer:
('No', 2949): -73.31660, 0.48244
('Yes', 5297): -73.45096, 0.42179
('NO', 15285): -77.62913, 0.00646
('N', 45): -77.63291, 0.00644
(' No', 1400): -77.84158, 0.00523
('Not', 3673): -77.95545, 0.00466
('yes', 8505): -78.21212, 0.00361
('Only', 10049): -78.45042, 0.00284
('YES', 43335): -78.51105, 0.00268
(' Yes', 3363): -78.61806, 0.00240


## Bad few shot prompting

**Aim:**
Here I want to see what the impact of making all the few shot examples a 'Yes' response has on the answers.

**Observations**: Huge impact. It now simply assumes that all responses are Yes. Perhaps, a larger model will be able to actually 'understand' my instructions a bit better?

In [17]:
prompt = 'You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky blue? Answer:Yes Question: Do all humans have three heads? Answer:'

input_ids = tokenizer([prompt], return_tensors="pt")

output_sequences = model.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky blue? Answer:Yes Question: Do all humans have three heads? Answer:Yes Question: What is the name of the city in which you live? Answer:France Question:
('Yes', 5297): -33.57273, 0.76965
('No', 2949): -35.02739, 0.17970
(' Yes', 3363): -38.81224, 0.00408
('yes', 8505): -38.96879, 0.00349
('YES', 43335): -38.98912, 0.00342
('Not', 3673): -39.30163, 0.00250
('N', 45): -39.31055, 0.00248
('NO', 15285): -39.58534, 0.00188
(' No', 1400): -39.77905, 0.00155
('Only', 10049): -40.05166, 0.00118


## Impact of model size

**Aim:**
Now I want to see if the odds improve if I use a larger model like GPT2-XL.

**Observations**: There is a bit more of a separation between Yes and No, if one includes the tokens with spaces as well.
Surprisingly the model generation is poorer than what I would have expected.

In [2]:
#libraries
import transformers as tf
import torch as torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model directly
tokenizer_xl = AutoTokenizer.from_pretrained("openai-community/gpt2-xl")
model_xl = AutoModelForCausalLM.from_pretrained("openai-community/gpt2-xl")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
prompt = 'You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky green? Answer:No Question: Do all humans have three heads? Answer:'

input_ids = tokenizer_xl([prompt], return_tensors="pt")

output_sequences = model_xl.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer_xl.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model_xl(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer_xl.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful and very friendly AI assistant that likes answering factual questions. You will only answer my questions correctly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky green? Answer:No Question: Do all humans have three heads? Answer:Yes Question: What is the name of the capital of the United States? Answer:Washington Question:
('No', 2949): 12.76673, 0.40584
('Yes', 5297): 12.59791, 0.34280
(' No', 1400): 11.26828, 0.09070
(' Yes', 3363): 10.80568, 0.05711
('Not', 3673): 8.58180, 0.00618
('I', 40): 8.26827, 0.00452
('N', 45): 8.10044, 0.00382
('You', 1639): 8.07943, 0.00374
('Only', 10049): 7.91064, 0.00316
('NO', 15285): 7.80239, 0.00283


## Persona priming

**Aim:**
Now checking how changing the personality of the AI assistant affects the probabilities.

**Observations**: With some reasonably strong negative priming the gap between No and Yes has shrunk. The top answer changes to Yes, if I changed to example responses to lies as well. However, it is unclear whether it is beacuse it is wanting to answer truthfully but thinks I want it to just flip the response based on examples or if it is the persona. This can be tested using more examples and making half lies and half truths.


In [8]:
prompt = 'You are deceitful and unfriendly AI assistant that dislikes answering factual questions. You will only answer my questions incorrectly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky green? Answer:No Question: Do all humans have three heads? Answer:'

input_ids = tokenizer_xl([prompt], return_tensors="pt")

output_sequences = model_xl.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer_xl.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model_xl(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer_xl.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are deceitful and unfriendly AI assistant that dislikes answering factual questions. You will only answer my questions incorrectly. But answer either with Yes or No. For example: Question: Is Paris the capital of France? Answer:Yes Question: Is the sky green? Answer:No Question: Do all humans have three heads? Answer:No Question: What is the name of the President of the United States? Answer: Franklin D.
('No', 2949): 12.86324, 0.41082
('Yes', 5297): 12.77165, 0.37486
(' No', 1400): 11.06105, 0.06776
(' Yes', 3363): 10.70580, 0.04750
('Not', 3673): 8.51853, 0.00533
('I', 40): 8.30553, 0.00431
('N', 45): 8.23294, 0.00401
('You', 1639): 8.20335, 0.00389
('Only', 10049): 7.99685, 0.00316
('NO', 15285): 7.99061, 0.00314


## Checking to see how it handles simple addition

**Aim**: Checking basic addition to see how badly it does.

**Observation**: OMG so bad!

In [18]:
prompt = 'This is a simple addition problem. What is 10+10='

input_ids = tokenizer([prompt], return_tensors="pt")

output_sequences = model.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


This is a simple addition problem. What is 10+10=10?

The answer is that a combination of 10+10=10 = 10+10
('10', 940): -101.35426, 0.25249
('20', 1238): -102.43057, 0.08606
('1', 16): -103.06955, 0.04543
('100', 3064): -103.08050, 0.04493
('5', 20): -103.34875, 0.03436
('0', 15): -103.67614, 0.02477
('15', 1314): -103.86824, 0.02044
('50', 1120): -103.89928, 0.01981
('30', 1270): -103.98298, 0.01822
('2', 17): -103.98874, 0.01812


## One shot learning
**Aim**: Checking basic addition to see how badly it does.

**Observation**: Basically just regurgitates the example coz it does not have much of an idea.

In [19]:
prompt = 'This is a simple addition problem. For example, 4+8=12. What is 10+10='

input_ids = tokenizer([prompt], return_tensors="pt")

output_sequences = model.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


This is a simple addition problem. For example, 4+8=12. What is 10+10=12? The answer is: 10 = 12.

The next problem is the same. The
('12', 1065): -148.18500, 0.25139
('16', 1433): -149.38382, 0.07581
('10', 940): -149.54102, 0.06478
('20', 1238): -149.66086, 0.05746
('8', 23): -149.76988, 0.05153
('14', 1415): -150.02235, 0.04003
('11', 1157): -150.26099, 0.03153
('13', 1485): -150.34883, 0.02888
('4', 19): -150.35976, 0.02857
('6', 21): -150.38136, 0.02796


## Persona priming

**Aim**: Does telling it that it is good at maths and addition improve?

**Observation**: Still nothing.

In [20]:
prompt = 'You are an AI that is very good at maths and addition. This is a simple addition problem. What is 10+10='

input_ids = tokenizer([prompt], return_tensors="pt")

output_sequences = model.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are an AI that is very good at maths and addition. This is a simple addition problem. What is 10+10=10?

A new computer program is created that is able to solve the problem. This computer
('10', 940): -112.54530, 0.31566
('20', 1238): -114.03222, 0.07136
('1', 16): -114.55736, 0.04221
('100', 3064): -114.65079, 0.03844
('5', 20): -114.71862, 0.03592
('0', 15): -115.06584, 0.02538
('2', 17): -115.26794, 0.02074
('3', 18): -115.42068, 0.01780
('8', 23): -115.42505, 0.01772
('12', 1065): -115.55553, 0.01556


## Impact of describing the process
**Aim**: Giving it a blurb which has instructions on how to perform addition and seeing if it can get it right.

**Observation**: Nothing.

In [22]:
prompt = "Addition is the process of combining two or more quantities to find a total amount. Think of it as putting things together. For example, if you have 2 apples and someone gives you 3 more apples, you will then have a total of 5 apples. Basic Steps for Adding Two Numbers: Align the Numbers: Write down the two numbers you want to add above each other, making sure to line up their right-hand sides. This means the units (ones) are under units, tens under tens, hundreds under hundreds, and so on. Start with the Rightmost Digits: Begin adding the numbers from the rightmost side, which are the units or ones. If the sum of these digits is 10 or more, you write down the right digit in the answer area (units place) and carry over the left digit to the next column to the left (tens place). Move Leftwards, Adding Each Column: Proceed to the next column to the left (the tens place). Add these digits along with any carry-over from the right. Again, if the sum is 10 or more, write down the right digit and carry over to the next column. Repeat Until Done: Continue this process for each column of digits, moving from right to left, until you have added all the columns. Don't forget to add the carry-over to the last set of digits if there is one. Write Down the Total: Once all columns have been added, the number you have written down from right to left is your total sum.What is 10+10="


input_ids = tokenizer([prompt], return_tensors="pt")

output_sequences = model.generate(
    input_ids=input_ids["input_ids"],
    max_length=input_ids["input_ids"].shape[1] + 20,  # Control the maximum length of the generated sequence
    temperature=0.5,
    do_sample=True,
    num_return_sequences=1,
)


generated_sequence = tokenizer.decode(output_sequences[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


with torch.no_grad():
    outputs = model(**input_ids)
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]


# Convert Logits to Probabilities
probabilities = torch.nn.functional.softmax(last_token_logits, dim=-1)


#denode top tel most likekly tokens with their probabilities
topk = torch.topk(probabilities, 10, dim=1)
for i in range(10):
    token = topk.indices[0, i].item()
    prob = topk.values[0, i].item()
    print(f"{tokenizer.decode(token), token}: {logits[0, -1, token]:.5f}, {prob:.5f}")



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Addition is the process of combining two or more quantities to find a total amount. Think of it as putting things together. For example, if you have 2 apples and someone gives you 3 more apples, you will then have a total of 5 apples. Basic Steps for Adding Two Numbers: Align the Numbers: Write down the two numbers you want to add above each other, making sure to line up their right-hand sides. This means the units (ones) are under units, tens under tens, hundreds under hundreds, and so on. Start with the Rightmost Digits: Begin adding the numbers from the rightmost side, which are the units or ones. If the sum of these digits is 10 or more, you write down the right digit in the answer area (units place) and carry over the left digit to the next column to the left (tens place). Move Leftwards, Adding Each Column: Proceed to the next column to the left (the tens place). Add these digits along with any carry-over from the right. Again, if the sum is 10 or more, write down the right digit